# Subset processes

The WPS flyingpigeon provides several processes to perform spatial subsetts of netCDF files. 

This are:

* **bounding box:** 
reduces the input netCDF files to a given latitude longitude bounding box. 

* **country or continent subset:** 
to reduce netCDF files to only the intersection of selected polygons. Countries and Continents are predifined. 

* **WFS subset:** 
reduce netCDF files to only the intersection of given polygons available on a given WFS server. 


In [1]:
# import the WPS client and connet to the server
from birdy import WPSClient
import birdy 

# to display Images from an url
from IPython.core.display import HTML 
from IPython.display import Image

# download files stored in a metalink
from metalink import download

fp_server = 'http://localhost:8093/wps'   # link to the flyingpigoen server

# simple connection (not recommended for larger processing)
fp = WPSClient(fp_server)

# asyncron connection with progess status requests
fp_i = WPSClient(url=fp_server, progress=True)

Explore the available processes: 

In [2]:
# fp? for general exploration on processes provided by flyingpigeon
fp?

# of check out a process in detail:
help(fp.subset_continents) # or type: fp.subset_countries?

Help on method subset_continents in module birdy.client.base:

subset_continents(resource=None, region='Africa', mosaic=False) method of birdy.client.base.WPSClient instance
    Return the data whose grid cells intersect the selected continents for each input dataset.
    
    Parameters
    ----------
    region : {'Africa', 'Asia', 'Australia', 'North America', 'Oceania', 'South America', 'Antarctica', 'Europe'}string
        Continent name.
    mosaic : boolean
        If True, selected regions will be merged into a single geometry.
    resource : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-tar`, :mimetype:`application/zip`
        NetCDF Files or archive (tar/zip) containing netCDF files.
    
    Returns
    -------
    output : ComplexData:mimetype:`application/x-netcdf`
        NetCDF output for first resource file.
    metalink : ComplexData:mimetype:`application/metalink+xml; version=4.0`
        Metalink file with links to all NetCDF outputs.



In [3]:
# point out some input files: 

url1 = 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep.reanalysis.dailyavgs/surface/slp.2000.nc'
url2 = 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep.reanalysis.dailyavgs/surface/slp.2001.nc'
url3 = 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep.reanalysis.dailyavgs/surface/slp.2002.nc'
url4 = 'https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep.reanalysis.dailyavgs/surface/slp.2003.nc' 

#### Subtset a bounding box 

All netCDF input files will be subsetted seperately

In [4]:
out = fp_i.subset_bbox(
            resource=[url1,url2,], 
            lon0=20,
            lon1=70,
            lat0=10,
            lat1=50,
            start=None,  # optional to select a time periode
            end=None,
            variable=None,  # can be set if the variable is known, otherwise the process detects the variable name 
            )

# You need to wait until the processing is complete!

In [5]:
# check the output files:
out.get()

subset_bboxResponse(
    output='http://127.0.0.1:8093/outputs/3d2add84-6922-11ea-9223-9cb6d08a53e7/slp.2000_bbox_subset.nc',
    metalink='http://127.0.0.1:8093/outputs/3d2add84-6922-11ea-9223-9cb6d08a53e7/input.meta4'
)

In [6]:
# plot the test file with the flyingpigon plot function 
out_plot = fp.plot_map_timemean(resource=out.get()[0])

# and display the output
Image(url= out_plot.get()[0], width=400)

In [9]:
# if you want to download the result files: 

dir_output = '/home/nils/data/'
files = download.get(out.get()[1], path=dir_output, segmented=False)

Metalink content-type detected.


#### Call a continent subset process

All netCDF input files will be subsetted seperatly and depending on mosic=True or Fales the selected polygons are given as seperated files or one output file per input file including one intersection of all selected polygons. 

**subset_countries** is working in the same principe

In [10]:
# run the process
out = fp_i.subset_continents(resource=[url1, url2, url3, url4], region=['Europe', 'Africa'], mosaic=True)

# You need to wait until the processing is done!

There are two outputs: 
* a netCDF file to have a quick test to check if the process went according to the users needs
* a metalink file with the list of all output files

In [11]:
# check the output files:
out.get()

subset_continentsResponse(
    output='http://127.0.0.1:8093/outputs/c8f11026-6923-11ea-9aa4-9cb6d08a53e7/slp.2000_EuropeAfrica.nc',
    metalink='http://127.0.0.1:8093/outputs/c8f11026-6923-11ea-9aa4-9cb6d08a53e7/input.meta4'
)

In [12]:
# plot the test file with the flyingpigon plot function 
out = fp_i.plot_map_timemean(resource=out.get()[0])

In [13]:
# the plot process returnes one graphic file
out.get()

plot_map_timemeanResponse(
    plotout_map='http://127.0.0.1:8093/outputs/40053b56-6924-11ea-9811-9cb6d08a53e7/tmpbcimpf5v.png'
)

In [14]:
Image(url= out.get()[0], width=400)